In [ ]:
# conda envs --info
# conda activate facialrecog
# Procurar rostos dentro de uma imagem, recortar o rosto e salvar o rosto dentro de uma pasta 

from PIL import Image
import face_recognition
import os
import re


# Procurar rostos em imagens dentro de uma pasta: faces_folder
# [os.path.join(folder, f) for f in os.listdir(folder) if re.match(r'.*\.(jpg|jpeg|png)', f, flags=re.I)]

# Escanear imagem e encontrar rostos
image = face_recognition.load_image_file("/home/raime/Documentos/Repositórios/cam_face_recognition_app/face_recognition/Algoritmo/Data/to_test/teste_hehe.jpeg")
face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="cnn")

print("Eu encontrei {} rostos nessa imagem".format(len(face_locations)))

# contagem de rostos dentro da imagem
count = 1

for face_location in face_locations:

    # Print o local de cada rosto nesta imagem
    top, right, bottom, left = face_location
    print("O(s) resto(s) estão localizados nos pixels Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # Visualização recortada dos rostos
    face_image = image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    pil_image.show()

    # Salvando em uma pasta para posterior reconhecimento facial
    save_dir = '/home/raime/Documentos/Repositórios/cam_face_recognition_app/face_recognition/Algoritmo/Data/to_recog'
    nome_input = input('Qual o nome desta pessoa?')
    pil_image.save(f'{save_dir}/{nome_input}.jpeg')
    count += count + 1  


# Reconhecimento facial da pasta dos rostos recortados: face_recog_folder. Chamar a função scan_know_people

def image_files_in_folder(folder='/home/raime/Documentos/Repositórios/cam_face_recognition_app/face_recognition/Algoritmo/Data/to_recog'):
    return [os.path.join(folder, f) for f in os.listdir(folder) if re.match(r'.*\.(jpg|jpeg|png)', f, flags=re.I)]

def scan_known_people(known_people_folder):
    known_names = []
    known_face_encodings = []

    for file in image_files_in_folder(known_people_folder):
        basename = os.path.splitext(os.path.basename(file))[0]
        img = face_recognition.load_image_file(file)
        encodings = face_recognition.face_encodings(img)

        if len(encodings) > 1:
            click.echo("WARNING: Mais de um rosto encontrado em {}. Considerar somente o primeiro rosto.".format(file))

        if len(encodings) == 0:
            click.echo("WARNING: Não foram encontrados rostos em {}. Ignorando arquivo.".format(file))
        else:
            known_names.append(basename)
            known_face_encodings.append(encodings[0])

    return known_names, known_face_encodings


In [ ]:
# Guardar na variável coração do pipeline os nomes e os numpy arrays gerados
heart_pipeline = scan_known_people(known_people_folder='/home/raime/Documentos/Repositórios/cam_face_recognition_app/face_recognition/Algoritmo/Data/to_recog')


In [ ]:
# Reconhecimento facial a partir da captura de imagem em real-time a partir da dettecção e recorte de rostos

import face_recognition
import cv2
import numpy as np

# Este é um exemplo super simples (mas lento) de como executar o reconhecimento de rosto em vídeo ao vivo de sua webcam.

# ATENÇÃO: Este exemplo requer que o OpenCV (a biblioteca `cv2`) seja instalado apenas para leitura de sua webcam.
# OpenCV * não * é necessário para usar a biblioteca face_recognition. É necessário apenas se você quiser executar esta
# demonstração específica. Se você tiver problemas para instalá-lo, tente qualquer um dos outros demos que não o requeiram.

# Obtenha uma referência para a webcam #0 (padrão)

video_capture = cv2.VideoCapture(0)

# Crie matrizes de codificações de rosto conhecidas e seus nomes

known_face_encodings = heart_pipeline[1]

known_face_names = heart_pipeline[0]

while True:
    # Pegue um único quadro de vídeo
    ret, frame = video_capture.read()

    # Converta a imagem da cor BGR (que o OpenCV usa) para a cor RGB (que usa o face_recognition)
    rgb_frame = frame[:, :, ::-1]

    # Encontre todos os rostos e codificações de rosto no quadro do vídeo
    video_face_locations = face_recognition.face_locations(rgb_frame)
    video_face_encodings = face_recognition.face_encodings(rgb_frame, video_face_locations)

    # Faça um loop em cada rosto neste quadro de vídeo
    for (top, right, bottom, left), video_face_encoding in zip(video_face_locations, video_face_encodings):
        
        # Veja se o rosto é compatível com o(s) rosto(s) conhecido(s)
        matches = face_recognition.compare_faces(known_face_encodings, video_face_encoding)

        name = "Desconhecido"

        # Se uma correspondência foi encontrada em known_face_encodings, apenas use a primeira.
        if True in matches:
             first_match_index = matches.index(True)
             name = known_face_names[first_match_index]

        # Ou em vez disso, use o rosto conhecido com a menor distância para o novo rosto
        #face_distances = face_recognition.face_distance(known_face_encodings, video_face_encoding)
        #best_match_index = np.argmin(face_distances)
        
        #if matches[best_match_index]:
        #    name = known_face_names[best_match_index]

        # Desenhe uma caixa ao redor do rosto

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Desenhe uma etiqueta com um nome abaixo do rosto

        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Mostra a imagem resultante
    cv2.imshow('Video', frame)

    # Pressione 'q' no teclado para sair!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Solte o enquadramento para a webcam
video_capture.release()
cv2.destroyAllWindows()